In [ ]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [ ]:
#open the browser
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
time.sleep(3)
driver.maximize_window()
wait_time = WebDriverWait(driver, 20)

def get_himachal_bus_data(xpath):
    bus_links = []  # Holds all the bus URLs
    bus_routes = []  # Holds all the route names
    
    for page_num in range(1, 5):
        route_elements = driver.find_elements(By.XPATH, xpath)
        bus_links.extend([element.get_attribute("href") for element in route_elements])
        bus_routes.extend([element.text for element in route_elements])
        
        try:
            pagination_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]'))
            )
            next_page_button = pagination_table.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{page_num + 1}"]')
            
            # Scroll the next page button into view
            ActionChains(driver).move_to_element(next_page_button).perform()
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_page_button))
            next_page_button.click()
            
        except NoSuchElementException:
            print(f"End of pages detected at page {page_num}")
            break
        except ElementClickInterceptedException:
            print(f"Could not click the next page button at page {page_num}, trying again.")
            time.sleep(1)  # Wait a moment before retrying
            next_page_button.click()
    
    return bus_links, bus_routes

In [3]:
# Run the function to get the bus links and routes
bus_links, bus_routes = get_himachal_bus_data("//a[@class='route']")

# Now create the DataFrame with the returned variables
himachal_df = pd.DataFrame({
    "Bus_Route_Name": bus_routes,  # variable name from ROUTE
    "Bus_Route_Link": bus_links   # variable name from LINKS
})

# Display the DataFrame
print(himachal_df)


End of pages detected at page 4
                                   Bus_Route_Name  \
0                                 Delhi to Shimla   
1                                 Shimla to Delhi   
2       Chandigarh to Hamirpur (Himachal Pradesh)   
3       Hamirpur (Himachal Pradesh) to Chandigarh   
4                             Delhi to Chandigarh   
5                            Manali to Chandigarh   
6                            Shimla to Chandigarh   
7                            Chandigarh to Manali   
8                                Shimla to Manali   
9    Dharamshala (Himachal Pradesh) to Chandigarh   
10                                Delhi to Manali   
11   Chandigarh to Dharamshala (Himachal Pradesh)   
12        Chamba (Himachal Pradesh) to Chandigarh   
13                                Manali to Delhi   
14           Delhi to Hamirpur (Himachal Pradesh)   
15           Hamirpur (Himachal Pradesh) to Delhi   
16                                 Solan to Delhi   
17            

In [4]:
# change dataframe to csv
path=r"C:\Users\Vanathi\Documents\redbus scraped details\himachal_df.csv"
himachal_df.to_csv(path,index=False)

In [5]:
df=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\himachal_df.csv")

In [6]:
driver= webdriver.Chrome()
Busname_1 = []
Bustype_1 = []
Starttime_1 = []
Endtime_1 = []
Ratings_1 = []
Total_duration_1 = []
Prices_1 = []
Seats_available_1 = []
Route_names = []
Route_links = []

for i, r in df.iterrows():
    link = r["Bus_Route_Link"]
    routes = r["Bus_Route_Name"]

    # Open the link
    driver.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click elements to view bus details
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll through the page to load all bus details
    scrolling = True
    while scrolling:
        # Get the current scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(5)

        # Get the new scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Check if the page height has changed
        if new_height == last_height:
            scrolling = False

    # Extract bus details
    try:
        bus_name = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='travels lh-24 f-bold d-color']"))
        )
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Busname_1.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bustype_1.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Starttime_1.append(start_time_elem.text)
        for end_time_elem in end_time:
            Endtime_1.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_duration_1.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_1.append(ratings.text)
        for price_elem in price:
            Prices_1.append(price_elem.text)
        for seats_elem in seats:
            Seats_available_1.append(seats_elem.text)
    except Exception as e:
        print(f"Error occurred while extracting details: {e}")
        continue

print("Successfully Completed")

Successfully Completed


In [7]:
# Before converting to DataFrame, check if all lists have the same length
lengths = {
    'Busname_1': len(Busname_1),
    'Bustype_1': len(Bustype_1),
    'Starttime_1': len(Starttime_1),
    'Endtime_1': len(Endtime_1),
    'Total_duration_1': len(Total_duration_1),
    'Prices_1': len(Prices_1),
    'Seats_available_1': len(Seats_available_1),
    'Route_names': len(Route_names),
    'Route_links': len(Route_links),
    'Ratings_1': len(Ratings_1)
}

print(lengths)

# Ensure that all lists have the same length
max_length = max(lengths.values())  # Find the longest list
for lst in [Busname_1, Bustype_1, Starttime_1, Endtime_1, Total_duration_1, Prices_1, Seats_available_1, Route_names, Route_links, Ratings_1]:
    while len(lst) < max_length:
        lst.append(None)  # Or you can append a placeholder like an empty string


{'Busname_1': 816, 'Bustype_1': 816, 'Starttime_1': 816, 'Endtime_1': 816, 'Total_duration_1': 816, 'Prices_1': 816, 'Seats_available_1': 816, 'Route_names': 816, 'Route_links': 816, 'Ratings_1': 0}


In [11]:
# Convert list to DataFrame
data = {
    'Bus_name': Busname_1,
    'Bus_type': Bustype_1,
    'Start_time': Starttime_1,
    'End_time': Endtime_1,
    'Total_duration': Total_duration_1,
    'Price': Prices_1,
    "Seats_Available": Seats_available_1,
    "Ratings": Ratings_1,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_6= pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_6.csv"
df_buses_6.to_csv(path, index=False)

print("Successfully Completed and saved to CSV.")

Successfully Completed and saved to CSV.


In [12]:
f=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_6.csv")
f

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,RSRTC - 192344,Super Luxury Volvo AC Seater Pushback 2+2,03:00,14:00,11h 00m,INR 1117,16 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,Bedi Travels,Volvo A/C Semi Sleeper (2+2),23:00,08:00,09h 00m,1241,3 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),23:30,08:00,08h 30m,INR 1399,19 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,Zingbus Plus,Bharat Benz A/C Semi Sleeper (2+2),22:35,08:10,09h 35m,1117,13 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,Laxmi holidays,Scania Multi-Axle AC Semi Sleeper (2+2),22:30,07:05,08h 35m,INR 999,5 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
811,HRTC - 531,Ordinary 3+2 Non AC Seater,17:50,07:10,13h 20m,INR 790,36 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-dal...,Delhi to Dalhousie
812,HRTC - 84,Himsuta AC Seater Volvo/Scania 2+2,20:15,09:30,13h 15m,INR 1552,6 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-dal...,Delhi to Dalhousie
813,Zingbus Plus,A/C Sleeper (2+1),21:35,11:00,13h 25m,2933,13 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-dal...,Delhi to Dalhousie
814,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,09:05,13h 05m,2473,14 Seats available,NaN,https://www.redbus.in/bus-tickets/delhi-to-dal...,Delhi to Dalhousie
